In [70]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential  #layerlari lineer olarak bir araya getirir
from keras.layers import Dense, Embedding, LSTM  # Dense layerdaki noronlarin birbirine baglanmasini saglar
from keras.models import load_model
from keras.utils.np_utils import to_categorical
import re

In [71]:
turkish_characters = "a|b|c|ç|d|e|f|g|ğ|h|ı|i|j|k|l|m|n|o|ö|p|r|s|ş|t|u|ü|v|y|z|0-9"
data = pd.read_csv('equal_duygu.csv',sep=',')
for x in data['Yorum']:
    print(x)#data['Yorum'][0]

çarşamba günü sipariş verdim orda yaizıyor salı gunu gelcek hani 24 saate kargodaydı
1 yıl dayanmadı ıdare eder demıcem cunku etmedı heryerınde bı arza cıktı servısı cok ıyı onun harıncınde alt sevıye bır telefon genç işi deıl telefonu az kullananlar ıcın ıdael olabilir.
1 günde yozgattan hataya geldi hızlı kargo ve temiz alışveriş için teşekkür ederim gsmport
dunden beri bekliyorum hala kargolamadiniz
bayrama yetişsin dedik ama.dün sözde kargolayacaklardı.şimdi bile sipariş ver 3 saate kargoda demesine rağmen 2 gündür hala paketleme sürecindeymiş
Ürün geldiğinde ekranında problem olduğunu farkettim Samsung müşteri hizmetini aradım yardımcı olamadılar hepsiburada sorunumu çözdü telefonu iade aldı bende tekrardan sipariş verdim bu sefer sorunsuz geldi memnunum telefondan
Bu fiyata alınabilecek en iyi telefon kasa ekran heryeri plastik ama telefon sağlam çok fazla yete düştü ama birşey olmadı
Samsung Galaxy M20 32 GB (Samsung Türkiye Garantili... ıyi çok güzel
arkadaslar hepsiburadan gön

In [72]:
data = data[['Yorum','Duygu']]

In [73]:
#type(data[0])
type(data['Yorum'])

pandas.core.series.Series

In [74]:
# Veri Setinin temizlenmesi.
data['Yorum'] = data['Yorum'].apply(lambda x: x.lower())
data['Yorum'] = data['Yorum'].apply((lambda x: re.sub('[^'+turkish_characters+'\s]','',x)))


In [75]:
# Veri setimizin işlenebilmesi için text verileri numaralara çevirmemiz gerekir.
# Keras bu işlem için hazır bir mekanizma sunmaktadır.
# Tokenizer sınıfı data içerisinde verilen cümleleri analiz ederek. Kelimelerin sıklıklarını hesaplar.
# Parameter: num_words = En sık geçen 250 kelimeye odaklan. Diğerleri önemli değil
tokenizer = Tokenizer(split=' ',num_words=250)
# Her bir kelimenin sıklığını(frekansını) hesaplar.
tokenizer.fit_on_texts(data['Yorum'].values)
# Tüm cümleler tam sayı dizisine dönüştürülür.
X = tokenizer.texts_to_sequences(data['Yorum'].values)
# Bütün metinlerimiz 400 sütundan oluşan bir dizi ile temsil edilecek.
# Çok kısa metinler 0'lar ile doldurulacak. Çok uzun metinler ise kesilecek.
X = pad_sequences(X,maxlen=400)

In [79]:
embed_dim = 128
lstm_out = 128
def build_model():
    model = Sequential()
    # Her bir kelimenin temsil edileceği vektör boyutu. Bu örnek için her bir kelime 128 boyutunda
    # bir vektör ile temsil edilir.
    model.add(Embedding(2500, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
    model.add(Dense(128,activation='relu'))  # 2 noron sayisi
    model.add(Dense(2,activation='softmax'))  # 2 noron sayisi
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    #optimizer =  ogrenme oranini kontrol eder, ogrenme orarnin ne kadar hizli hesaplanancaginin kontrol eder
    #metrics = modelin ne kadar iyi tahmin yaptiginin soyler
    return model

In [80]:
# Çıktılarımızı kategorik hale getirdik. (Opsiyonel)
Y = pd.get_dummies(data['Duygu']).values
Y

array([[0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0,

In [82]:
# Verinin %80'i train, %20'si test verisi olacak şekilde ayrılır.
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.1, random_state = 2)

model = build_model()
# Oluşturulan model train verileri ile eğitilir. Yapay Sinir Ağı eğitilmeye başlar.
# nb_epoch: İterasyon sayısı
#batch_size : model egitilirken kacar orenek alinacaginin soyler
model.fit(X_train, Y_train, epochs = 5, batch_size=64, verbose = 2)
# Train verileri ile model eğitildikten sonra test dataları ile doğruluk oranlarına bakılır.
score = model.evaluate(X_test, Y_test, verbose = 2)
print("score: %.2f" % (score[1]))

C:\Users\yasin\Anaconda3\envs\nlp_course\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, dropout=0.2, recurrent_dropout=0.2)`
  


Epoch 1/5
 - 6s - loss: 0.6893 - accuracy: 0.5889
Epoch 2/5
 - 4s - loss: 0.6681 - accuracy: 0.6750
Epoch 3/5
 - 4s - loss: 0.6323 - accuracy: 0.6972
Epoch 4/5
 - 5s - loss: 0.5588 - accuracy: 0.7694
Epoch 5/5
 - 5s - loss: 0.4325 - accuracy: 0.8361
score: 0.82


In [83]:
X_train.shape , X_test.shape

((360, 400), (40, 400))

In [84]:
Y_train.shape , Y_test.shape

((360, 2), (40, 2))

In [85]:
X_train.var(), X_test.var()

(336.8215073772665, 275.92407810937505)

In [86]:
# Save Model
model.save("equal_duygu_model.h5")


In [87]:
# Load Model. Daha önce eğitilmiş olan model. Veriseti yada model parametreleri değişirse
# bu model geçersiz olur. Eğitim işlemi uzun sürdüğü için ağı bir defa eğitip oluşan modeli kaydettim.
model = load_model("equal_duygu_model.h5")

In [88]:
my_text = ["çarşamba günü sipariş verdim orda yaizıyor salı gunu gelcek hani 24 saate kargodaydı",
           "bozuk",
           " hiç beğenmedim",
           "süper",
           "kötü" 
           "daha iyi olabilir",
           "Bugun kargoya verilmesi gerekirken satıcı kargolama yapmamış.",
           "Telefon çok güzel herkese öneririm",
           "kamerası kötü görünüşü iyi diye aldık yanıldım"]

# Verilen örnekler Tokenizer yapısı ile tam sayı dizisine dönüştürülür
# Daha sonra eğitilen modele sırayla verilerek anlam analizi sonuçları elde edilir.
# Her Cümlenin yüzde kaç olumlu ve olumsuz olduğuna dair bilgiler çıktı olarak verilir.
sequences = tokenizer.texts_to_sequences(my_text)
data = pad_sequences(sequences, maxlen=400)
predictions = model.predict(data)
print(predictions)

[[0.34048817 0.6595118 ]
 [0.2784126  0.7215874 ]
 [0.46457684 0.53542316]
 [0.17528081 0.8247192 ]
 [0.16951801 0.830482  ]
 [0.2784126  0.7215874 ]
 [0.13657196 0.863428  ]
 [0.41491565 0.5850843 ]]
